In [13]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Activation, Dropout, Dense, Conv1D, GlobalMaxPool1D, MaxPool1D, Input, MaxPooling1D, Flatten, SimpleRNN
from tensorflow.keras.models import Model, Sequential
from keras.backend import clear_session
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.activations import tanh
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
from tensorflow import keras

In [5]:
dataset = pickle.load(open("dataset/IMDB/dataset_IMDB.pickle", "rb"))

In [6]:
Y = np.array(list(map(lambda x: 1 if x=="positive" else 0, dataset["sentiment"])))

In [7]:
x_train,x_test,y_train,y_test = train_test_split(dataset["processed_review"],Y, test_size=0.1, shuffle=True)

In [8]:
def creazione_modello_GloVe(filename):
    f = open(filename, encoding="utf8")
    embeding_index = {}
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embeding_index[word] = coefs
    f.close()
    return embeding_index

In [9]:
embedding = creazione_modello_GloVe("dataset/glove.6B.50d.txt")

In [10]:
tokenizer = Tokenizer(num_words=142092)
tokenizer.fit_on_texts(x_train)

word_index = tokenizer.word_index

In [11]:
vocab_len = len(word_index)+1
embedding_vector_len = embedding["banan"].shape[0]
embedding_matrix = np.zeros((vocab_len, embedding_vector_len))

for word, index in word_index.items():
    vector = embedding.get(word)
    if vector is not None:
        embedding_matrix[index, :] = vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embedding_vector_len, input_length=200, weights=[embedding_matrix])

In [44]:
CNNRNN_model = Sequential()
CNNRNN_model.add(embedding_layer)
#CNN
CNNRNN_model.add(Conv1D(filters=4, kernel_size=8, padding="same"))
CNNRNN_model.add(MaxPooling1D(5, padding="same"))
#CNNRNN_model.add(Flatten())
#RNN
CNNRNN_model.add(SimpleRNN(64))
CNNRNN_model.add(Dense(64))
CNNRNN_model.add(Dense(64))
#CNN_model.add(Dropout(0.2))
#CNN_model.add(Dense(20))
CNNRNN_model.add(Dense(2, activation=tanh))
#RMSprop(learning_rate=0.20)
CNNRNN_model.compile(loss="binary_crossentropy", optimizer=Adagrad(), metrics=["accuracy"])
CNNRNN_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 50)           5019350   
_________________________________________________________________
conv1d (Conv1D)              (None, 200, 4)            1804      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 40, 4)             0         
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 64)                4416      
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1

In [24]:
x_train_index = tokenizer.texts_to_sequences(x_train)
x_train_index = pad_sequences(x_train_index, maxlen=200)

In [25]:
y_train_categorical = keras.utils.to_categorical(y_train, 2)

In [43]:
clear_session()
CNNRNN_model.fit(x_train_index[:35000], y_train_categorical[:35000], epochs=10, batch_size=64, verbose=1, validation_data=(x_train_index[35000:], y_train_categorical[35000:]))

Epoch 1/10
547/547 [==============================] - 13s 22ms/step - loss: 0.8127 - accuracy: 0.5095 - val_loss: 0.7242 - val_accuracy: 0.5280
Epoch 2/10
547/547 [==============================] - 12s 22ms/step - loss: 0.7148 - accuracy: 0.5324 - val_loss: 0.7036 - val_accuracy: 0.5368
Epoch 3/10
547/547 [==============================] - 12s 22ms/step - loss: 0.7008 - accuracy: 0.5435 - val_loss: 0.6950 - val_accuracy: 0.5456
Epoch 4/10
264/547 [=============>................] - ETA: 6s - loss: 0.6950 - accuracy: 0.5529

KeyboardInterrupt: 

In [32]:
x_test_index = tokenizer.texts_to_sequences(x_test)
x_test_index = pad_sequences(x_test_index, maxlen=200)

In [33]:
y_pred = CNNRNN_model.predict(x_test_index)

In [34]:
print(classification_report(y_test, np.argmax(y_pred, axis=1).astype("float32")))

              precision    recall  f1-score   support

           0       0.72      0.69      0.71      2496
           1       0.71      0.74      0.72      2504

    accuracy                           0.71      5000
   macro avg       0.72      0.71      0.71      5000
weighted avg       0.71      0.71      0.71      5000

